In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob
print(datetime.datetime.now())
os.getcwd()

2020-01-28 13:38:24.412921


'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Segment_Broyhill'

In [2]:
df_product_selected=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Segment_Broyhill/Bryohill SKU List_and_2classes_20200116.xlsx",
                                 sheet_name="selected_items",dtype=str)
print(df_product_selected.shape)
df_product_selected=df_product_selected.rename(columns={"Item":"item_id"})
df_product_selected['product_type']=np.where(df_product_selected['Division']=="9","Broyhill",
                                            df_product_selected['Class'])
df_product_selected['product_type']=df_product_selected['product_type'].replace("21012","Patio")
df_product_selected['product_type']=df_product_selected['product_type'].replace("21034","Gazebos")
df_product_selected=df_product_selected[df_product_selected['product_type'].isin(['Broyhill','Patio','Gazebos'])]

df_product_selected=df_product_selected[['product_type','item_id']]
print(df_product_selected.shape)
df_product_selected.groupby("product_type")['item_id'].count().to_frame()

(39, 10)
(39, 2)


,item_id
product_type,
Broyhill,22
Gazebos,4
Patio,13


In [3]:
prod_taxonomy=pd.read_table("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20200101-135600-916.txt",
                           sep="|",dtype=str)

prod_taxonomy=prod_taxonomy[['division_id','class_code_id','subclass_id']].drop_duplicates()
prod_taxonomy.shape

(2875, 3)

In [4]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)
            
date_last_Saturday_ty=datetime.date(2020,1,18)
dates_19Q4_start_ty=datetime.date(2019,11,3)

dates_18Q4_start_ly=dates_19Q4_start_ty-datetime.timedelta(days=52*7)
date_18Q4_end_ly=datetime.date(2019,2,2)


list_daily_sales_ty=list(recursive_file_gen("/home/jian/BigLots/"))
list_daily_sales_ty=[x for x in list_daily_sales_ty if "daily" in x.lower() and "/MediaStorm_" in x]
list_daily_sales_ty=[x for x in list_daily_sales_ty if x.split("/MediaStorm_")[1][:10]>=str(dates_19Q4_start_ty)]
list_daily_sales_ty.sort()
print(len(list_daily_sales_ty))
######
list_daily_sales_ly=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")

list_daily_sales_ly=[x for x in list_daily_sales_ly if x.split("ailySalesHistory")[1][:8]>=str(dates_18Q4_start_ly).replace("-","")]
list_daily_sales_ly=[x for x in list_daily_sales_ly if x.split("ailySalesHistory")[1][:8]<=str(date_18Q4_end_ly).replace("-","")]
list_daily_sales_ly.sort()
print(len(list_daily_sales_ly))

12
13


In [5]:
k_sample=None
df_sales_detail_both_year=pd.DataFrame()

i_counter=0
for file in list_daily_sales_ty:
    
    df=pd.read_table(file,dtype=str,sep="|",nrows=k_sample)
    week_end_dt=df['transaction_dt'].max()
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df['item_transaction_units']=df['item_transaction_units'].astype(int)
    df=pd.merge(df,prod_taxonomy,on=['class_code_id','subclass_id'],how="left")
    df=pd.merge(df,df_product_selected,on=['item_id'],how="left")
    df['product_type']=np.where(pd.notnull(df['product_type']),df['product_type'],
                               np.where(df['division_id']=="9","other_furniture","0")
                               )
    df=df[df['product_type']!="0"]
    df['week_end_dt']=week_end_dt
    df_sales_detail_both_year=df_sales_detail_both_year.append(df)
    
    i_counter+=1
    print(i_counter,datetime.datetime.now())

1 2020-01-28 13:39:22.855553
2 2020-01-28 13:40:29.420938
3 2020-01-28 13:41:43.641270
4 2020-01-28 13:43:20.599833
5 2020-01-28 13:45:21.527504
6 2020-01-28 13:47:05.100255
7 2020-01-28 13:48:57.103291
8 2020-01-28 13:50:41.537218
9 2020-01-28 13:51:50.475071
10 2020-01-28 13:52:42.379200
11 2020-01-28 13:53:38.122341
12 2020-01-28 13:54:34.460106


In [6]:
k_sample=None
df_sales_detail_last_year=pd.DataFrame()

i_counter=0
for file in list_daily_sales_ly:
    df=pd.read_table(file,dtype=str,sep="|",nrows=k_sample)
    week_end_dt=df['transaction_dt'].max()
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df['item_transaction_units']=df['item_transaction_units'].astype(int)
    df=pd.merge(df,prod_taxonomy,on=['class_code_id','subclass_id'],how="left")
    df=pd.merge(df,df_product_selected,on=['item_id'],how="left")
    df['product_type']=np.where(pd.notnull(df['product_type']),df['product_type'],
                               np.where(df['division_id']=="9","other_furniture","0")
                               )
    df=df[df['product_type']!="0"]
    df['week_end_dt']=week_end_dt
    df_sales_detail_both_year=df_sales_detail_both_year.append(df)
    
    i_counter+=1
    print(i_counter,datetime.datetime.now())

1 2020-01-28 13:55:38.785433
2 2020-01-28 13:56:54.162435
3 2020-01-28 13:58:41.671247
4 2020-01-28 14:00:25.957593
5 2020-01-28 14:02:29.501233
6 2020-01-28 14:04:31.287209
7 2020-01-28 14:06:44.077196
8 2020-01-28 14:08:13.406998
9 2020-01-28 14:09:16.329690
10 2020-01-28 14:10:16.084497
11 2020-01-28 14:11:23.704874
12 2020-01-28 14:12:29.842558
13 2020-01-28 14:13:23.273841


In [7]:
df_sales_detail_both_year['rewards_label']=np.where(pd.isnull(df_sales_detail_both_year['customer_id_hashed']),"Non_rewards","Rewards")
df_sales_detail_both_year.to_csv("./BL_Broyhill_and_2_classes_13last_11this_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [8]:
df_by_day_sales=df_sales_detail_both_year.groupby(['rewards_label','product_type','week_end_dt'])['item_transaction_amt','item_transaction_units'].sum().reset_index()
df_by_day_sales=df_by_day_sales.rename(columns={"item_transaction_amt":"sales","item_transaction_units":"units"})

df_by_day_trans=df_sales_detail_both_year[['rewards_label','product_type','week_end_dt','location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
df_by_day_trans['trans']=1
df_by_day_trans=df_by_day_trans.groupby(['rewards_label','product_type','week_end_dt'])['trans'].sum().to_frame().reset_index()

df_by_day_shoppers=df_sales_detail_both_year[pd.notnull(df_sales_detail_both_year['customer_id_hashed'])]
df_by_day_shoppers=df_by_day_shoppers.groupby(['rewards_label','product_type','week_end_dt'])['customer_id_hashed'].nunique().to_frame().reset_index()
df_by_day_shoppers=df_by_day_shoppers.rename(columns={"customer_id_hashed":"shopper_count"})

df_output=pd.merge(df_by_day_sales,df_by_day_trans,on=['rewards_label','product_type','week_end_dt'],how="outer")
df_output=pd.merge(df_output,df_by_day_shoppers,on=['rewards_label','product_type','week_end_dt'],how="outer")
df_output['AOV_of_prodcut_type']=df_output['sales']/df_output['trans']

df_output=df_output.sort_values(["product_type","rewards_label","week_end_dt"])

In [20]:
df_output=df_output.fillna(0)

df_output_ty=df_output[df_output['week_end_dt']>"2019-11-01"]
df_output_ly=df_output[df_output['week_end_dt']<"2019-11-01"]

df_output_wide_ty=df_output_ty.pivot_table(index=['product_type','rewards_label'],columns="week_end_dt")
df_output_wide_ly=df_output_ly.pivot_table(index=['product_type','rewards_label'],columns="week_end_dt")

df_output_wide_ty=df_output_wide_ty.fillna(0)
df_output_wide_ly=df_output_wide_ly.fillna(0)


In [21]:
writer=pd.ExcelWriter("./BL_furniture_by_week_2_yearsQ4_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

df_output_wide_ty.to_excel(writer,"table_2019Q4")
df_output_wide_ly.to_excel(writer,"table_2018Q4")

df_output.to_excel(writer,"df_long",index=False)

writer.save()

In [24]:
df_sales_detail_both_year['product_type'].unique()

array(['other_furniture', 'Broyhill', 'Patio', 'Gazebos'], dtype=object)

In [25]:
df_sales_detail_both_year['item_transaction_units'].values[0]

1

In [29]:
price_by_item_lastyear.head(2)

,item_id,class_code_id,subclass_id,item_transaction_amt,item_transaction_units
0,610000321,61502,5,47.2,1
1,610000590,61004,4,15.0,1


In [31]:
df_sales_detail_this_year=df_sales_detail_both_year[df_sales_detail_both_year['transaction_dt']>="2019-11-01"]
price_by_item_thisyear=df_sales_detail_this_year.groupby(["item_id","class_code_id",'subclass_id'])['item_transaction_amt','item_transaction_units'].sum().reset_index()
price_by_item_thisyear['avg_price']=price_by_item_thisyear['item_transaction_amt']/price_by_item_thisyear['item_transaction_units']

df_sales_detail_last_year=df_sales_detail_both_year[df_sales_detail_both_year['transaction_dt']<"2019-11-01"]
price_by_item_lastyear=df_sales_detail_last_year.groupby(["item_id","class_code_id",'subclass_id'])['item_transaction_amt','item_transaction_units'].sum().reset_index()
price_by_item_lastyear['avg_price']=price_by_item_lastyear['item_transaction_amt']/price_by_item_lastyear['item_transaction_units']


In [32]:
writer=pd.ExcelWriter("./BL_item_avg_price_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
price_by_item_thisyear.to_excel(writer,"2019Q4",index=False)
price_by_item_lastyear.to_excel(writer,"2018Q4",index=False)
writer.save()


In [1]:
# Add the item name
import pandas as pd
import numpy as np
import datetime
import os
file_excel=pd.ExcelFile("./BL_item_avg_price_JL_2020-01-28.xlsx")
file_excel.sheet_names

['2019Q4', '2018Q4']

In [2]:
df_2019=file_excel.parse("2019Q4",dtype=str)
df_2018=file_excel.parse("2018Q4",dtype=str)


df_2019['item_transaction_amt']=df_2019['item_transaction_amt'].astype(float)
df_2019['item_transaction_units']=df_2019['item_transaction_units'].astype(int)
df_2019['avg_price']=df_2019['avg_price'].astype(float)

df_2018['item_transaction_amt']=df_2018['item_transaction_amt'].astype(float)
df_2018['item_transaction_units']=df_2018['item_transaction_units'].astype(int)
df_2018['avg_price']=df_2018['avg_price'].astype(float)

In [3]:
df_prod_taxo=pd.read_table("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20200101-135600-916.txt",
                          dtype=str,sep="|")
division_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",sep="|",dtype=str)
department_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Department Names.txt",sep="|",dtype=str)
class_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Class Names.txt",sep="|",encoding="iso-8859-1",dtype=str)


print(df_prod_taxo.shape)
df_prod_taxo=pd.merge(df_prod_taxo,division_name,on="division_id",how="left")
df_prod_taxo=pd.merge(df_prod_taxo,department_name,on="department_id",how="left")
df_prod_taxo=pd.merge(df_prod_taxo,class_name,on="class_code_id",how="left")
print(df_prod_taxo.shape)


(2875, 5)
(2875, 8)


In [4]:
df_2019=pd.merge(df_2019,df_prod_taxo,on=["class_code_id","subclass_id"],how="left")
df_2019=df_2019[['item_id','division_id','division_desc','department_id','department_desc','class_code_id','class_code_desc',
                 'subclass_id','subclass_desc','item_transaction_amt','item_transaction_units','avg_price']]

df_2018=pd.merge(df_2018,df_prod_taxo,on=["class_code_id","subclass_id"],how="left")
df_2018=df_2018[['item_id','division_id','division_desc','department_id','department_desc','class_code_id','class_code_desc',
                 'subclass_id','subclass_desc','item_transaction_amt','item_transaction_units','avg_price']]

In [9]:
file_excel_prod=pd.ExcelFile("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Segment_Broyhill/Bryohill SKU List_and_2classes_20200116.xlsx")
print(file_excel_prod.sheet_names)

df_product_selected=file_excel_prod.parse(sheet_name="selected_items",dtype=str)
print(df_product_selected.shape)
df_product_selected=df_product_selected.rename(columns={"Item":"item_id"})
df_product_selected['product_type']=np.where(df_product_selected['Division']=="9","Broyhill",
                                            df_product_selected['Class'])
df_product_selected['product_type']=df_product_selected['product_type'].replace("21012","Patio")
df_product_selected['product_type']=df_product_selected['product_type'].replace("21034","Gazebos")
df_product_selected=df_product_selected[df_product_selected['product_type'].isin(['Broyhill','Patio','Gazebos'])]
df_product_selected=df_product_selected[['product_type','item_id','Unnamed: 9']]
df_product_selected=df_product_selected.rename(columns={"Unnamed: 9":"item_name_selected"})
print(df_product_selected.shape)
df_product_selected.groupby("product_type")['item_id'].count().to_frame()


['selected_items', 'Broyhill SKUs']
(39, 10)
(39, 3)


,item_id
product_type,
Broyhill,22
Gazebos,4
Patio,13


In [6]:
df_2019=pd.merge(df_2019,df_product_selected,on="item_id",how="left")
df_2019['product_type']=df_2019['product_type'].fillna("other_furniture")
df_2019['item_name_selected']=df_2019['item_name_selected'].fillna("other_furniture")

df_2018=pd.merge(df_2018,df_product_selected,on="item_id",how="left")
df_2018['product_type']=df_2018['product_type'].fillna("other_furniture")
df_2018['item_name_selected']=df_2018['item_name_selected'].fillna("other_furniture")

df_2019=df_2019.sort_values(["division_id","product_type","item_id"])
df_2018=df_2018.sort_values(["division_id","product_type","item_id"])

In [7]:
writer=pd.ExcelWriter("./BL_item_price_Broyhil_and_Furniture_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_2019.to_excel(writer,"price_by_item_2019Q4_12W",index=False)
df_2018.to_excel(writer,"price_by_item_2018Q4_13W",index=False)
writer.save()
